In [1]:
import pandas as pd
data = pd.read_csv('dataset/data.csv')
data

,filepath,class,filename
0,datasets/asl_dataset/p/hand2_p_bot_seg_5_cropp...,P,hand2_p_bot_seg_5_cropped.jpeg
1,datasets/asl_dataset/p/hand3_p_dif_seg_5_cropp...,P,hand3_p_dif_seg_5_cropped.jpeg
2,datasets/asl_dataset/p/hand1_p_left_seg_3_crop...,P,hand1_p_left_seg_3_cropped.jpeg
3,datasets/asl_dataset/p/hand2_p_dif_seg_1_cropp...,P,hand2_p_dif_seg_1_cropped.jpeg
4,datasets/asl_dataset/p/hand2_p_dif_seg_2_cropp...,P,hand2_p_dif_seg_2_cropped.jpeg
...,...,...,...
120625,datasets/asl_alphabet_train/asl_alphabet_train...,P,P706.jpg
120626,datasets/asl_alphabet_train/asl_alphabet_train...,P,P2422.jpg
120627,datasets/asl_alphabet_train/asl_alphabet_train...,P,P42.jpg
120628,datasets/asl_alphabet_train/asl_alphabet_train...,P,P1490.jpg


In [2]:
from src.dataset import getdataset
dataset = getdataset(data)
dataset

Loaded 120630 files from 26 categories.


<BatchDataset shapes: ((None, 224, 224, 3), (None, 26, 26)), types: (tf.float32, tf.float32)>

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.applications.MobileNetV3Small(
    weights=None, classes=26
)

In [5]:
from keras.metrics import top_k_categorical_accuracy
from keras.losses import categorical_crossentropy, binary_crossentropy
from keras.optimizers import Adam
def top_5_accuracy(in_gt, in_pred):
    return top_k_categorical_accuracy(in_gt, in_pred, k=5)

LEARNING_RATE = 1e-4
model.compile(optimizer = Adam(lr=LEARNING_RATE), 
                   loss = 'categorical_crossentropy',
                   metrics = ['categorical_accuracy', top_5_accuracy])

In [6]:
ims, labels = next(iter(dataset.take(1)))

Prediction is at first just a uniform prediction, i.e. all weights equal.

In [7]:
model.predict(x=ims)[0] # first sample of some batch

array([0.03846154, 0.03846154, 0.03846154, 0.03846154, 0.03846154,
       0.03846154, 0.03846154, 0.03846154, 0.03846154, 0.03846154,
       0.03846154, 0.03846154, 0.03846154, 0.03846154, 0.03846154,
       0.03846154, 0.03846154, 0.03846154, 0.03846154, 0.03846154,
       0.03846154, 0.03846154, 0.03846154, 0.03846154, 0.03846154,
       0.03846154], dtype=float32)

In [8]:
ims.shape

TensorShape([32, 224, 224, 3])

In [9]:
labels.shape

TensorShape([32, 26, 26])

In [10]:
labels[0]

<tf.Tensor: shape=(26, 26), dtype=float32, numpy=
array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.,

In [11]:
model.test_on_batch(ims, y=labels)

ValueError: in user code:

    /home/s2995697/Documents/git/deep-learning/venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1233 test_function  *
        return step_function(self, iterator)
    /home/s2995697/Documents/git/deep-learning/venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1224 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/s2995697/Documents/git/deep-learning/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/s2995697/Documents/git/deep-learning/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/s2995697/Documents/git/deep-learning/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 

In [12]:
model.fit(x=dataset)

ValueError: in user code:

    /home/s2995697/Documents/git/deep-learning/venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/s2995697/Documents/git/deep-learning/venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/s2995697/Documents/git/deep-learning/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/s2995697/Documents/git/deep-learning/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/s2995697/Documents/git/deep-learning/venv/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _